### Knock21: 店舗を絞り込んで可視化できるようにしてみよう

In [1]:
# 注文 Data 4月分、店舗情報、Area 情報を結合
import pandas as pd
from IPython.display import display, clear_output

m_store = pd.read_csv('m_store.csv')
m_area = pd.read_csv('m_area.csv')
order_data = pd.read_csv('tbl_order_202004.csv')
order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')

# Master にない Code に対応した文字列を設定
order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = "デリバリー"
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = "お持ち帰り"

order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'

order_data.head()

order_id  store_id customer_id  coupon_cd  sales_detail_id  \
0  79339111        49   C26387220         50         67393872   
1  18941733        85   C48773811         26         91834983   
2  56217880        76   C66287421         36         64409634   
3  28447783       190   C41156423         19         73032165   
4  32576156       191   C54568117         71         23281182   

     order_accept_date       delivered_date  takeout_flag  total_amount  \
0  2020-04-01 11:00:00  2020-04-01 11:18:00             1          4144   
1  2020-04-01 11:00:00  2020-04-01 11:22:00             0          2877   
2  2020-04-01 11:00:00  2020-04-01 11:15:00             0          2603   
3  2020-04-01 11:00:00  2020-04-01 11:16:00             0          2732   
4  2020-04-01 11:00:00  2020-04-01 11:53:00             0          2987   

   status store_name area_cd wide_area narrow_area takeout_name status_name  
0       1        浅草店      TK        東京          東京        お持ち帰り        お支払済  
1       2        目黒店      TK        東京          東京        デリバリー        お渡し済  
2       2        本郷店      TK        東京          東京        デリバリー        お渡し済  
3       2        栃木店      TO       北関東          栃木        デリバリー        お渡し済  
4       2       伊勢崎店      GU       北関東          群馬        デリバリー        お渡し済

In [2]:
# ipywidgets の機能を実装し、Dropdown で店舗を選べるようにしていく
from ipywidgets import Dropdown


def order_by_store(val):
    clear_output()
    display(dropdown)
    pick_data = order_data.loc[
        (order_data['store_name'] == val['new'])
        & (order_data['status'].isin([1, 2]))
        ]
    display(pick_data.head())


store_list = m_store['store_name'].tolist()
dropdown = Dropdown(options=store_list)
dropdown.observe(order_by_store, names='value')
display(dropdown)

Dropdown(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛西店', '…

In [3]:
# 抽出した各店舗の注文 Data を可視化（折れ線 Graph 化）していく
import matplotlib.pyplot as plt
import japanize_matplotlib


def graph_by_store(val):
    clear_output()
    display(dropdown2)
    pick_data = order_data.loc[
        (order_data['store_name'] == val['new'])
        & (order_data['status'].isin([1, 2]))
        ]
    temp = pick_data[['order_accept_date', 'total_amount']].copy()
    temp.loc[:, 'order_accept_date'] = pd.to_datetime(temp['order_accept_date'])
    temp.set_index('order_accept_date', inplace=True)
    temp.resample('D').sum().plot()


dropdown2 = Dropdown(options=store_list)
dropdown2.observe(graph_by_store, names='value')
display(dropdown2)

C:\Users\leone\dsLab\machine_learning_system_100_knocks\.venv\lib\site-packages\japanize_matplotlib\japanize_matplotlib.py:15: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = font_manager.createFontList(font_files)


Dropdown(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛西店', '…

### Knock22: 複数店舗の詳細を可視化できるようにしてみよう
Select box で選んだ店舗の結果を表示していく。

In [4]:
from ipywidgets import SelectMultiple


def order_by_multi(val):
    clear_output()
    display(select)
    pick_data = order_data.loc[
        (order_data['store_name'].isin(val['new']))
        & (order_data['status'].isin([1, 2]))
        ]
    display(pick_data.head())


select = SelectMultiple(options=store_list)
select.observe(order_by_multi, names='value')
display(select)

SelectMultiple(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛…

In [5]:
# Multi select で Graph 描画
def graph_by_multi(val):
    clear_output()
    display(select2)

    fig = plt.figure(figsize=(17, 4))
    plt.subplots_adjust(wspace=0.25, hspace=0.6)

    i = 0

    for trg in val['new']:
        pick_data = order_data[
            (order_data['store_name'] == trg)
            & (order_data['status'].isin([1, 2]))
            ]
        temp = pick_data[['order_accept_date', 'total_amount', 'store_name']].copy()
        temp.loc[:, 'order_accept_date'] = pd.to_datetime(temp['order_accept_date'])
        temp.set_index('order_accept_date', inplace=True)
        i += 1
        ax = fig.add_subplot(1, len(val['new']), i)
        ax.plot(temp.resample('D').sum())
        ax.set_title(trg)


select2 = SelectMultiple(options=store_list)
select2.observe(graph_by_multi, names='value')
display(select2)

SelectMultiple(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛…

### Knock23: Slide-bar を用いて Order 件数を調べてみよう

In [6]:
# Slide-bar で閾値を設定し、閾値以下、以上で店舗を調べる
from ipywidgets import IntSlider


def store_lower(val):
    clear_output()
    display(slider)
    temp = order_data.groupby('store_name')
    print(temp.size()[temp.size() < val['new']])


slider = IntSlider(value=1100, min=1000, max=2000, step=100, description='件数:', )
slider.observe(store_lower, names='value')
display(slider)

IntSlider(value=1100, description='件数:', max=2000, min=1000, step=100)

In [7]:
# Slide-bar 件数上限を設ける
def store_upper(val):
    clear_output()
    display(slider2)
    temp = order_data.groupby('store_name')
    print(temp.size()[temp.size() > val['new']])


slider2 = IntSlider(value=1600, min=1000, max=2000, step=100, description='件数:', )
slider2.observe(store_upper, names='value')
display(slider2)

IntSlider(value=1600, description='件数:', max=2000, min=1000, step=100)

### Knock24: Toggle-button で地域 Data を抽出しよう
#### Toggle-button
- Data 切り替えに適している。
- List に似ているが、List より利便性が高い。
- 選択肢が多すぎると Button だらけになるので注意する。

In [8]:
# Toggle-button で地域毎に Data 抽出
from ipywidgets import ToggleButtons

area_list = m_area['wide_area'].unique()


def order_by_area(val):
    clear_output()
    display(toggle)
    pick_data = order_data.loc[
        (order_data['wide_area'] == val['new'])
        & (order_data['status'].isin([1, 2]))
        ]
    display(pick_data.head())


toggle = ToggleButtons(options=area_list)
toggle.observe(order_by_area, names='value')
display(toggle)

ToggleButtons(options=('東京', '神奈川', '千葉', '埼玉', '北関東'), value='東京')

In [9]:
# 地域ごとの order 情報を Graph にしてみる
def graph_by_area(val):
    clear_output()
    display(toggle2)
    pick_data = order_data.loc[
        (order_data['wide_area'] == val['new'])
        & (order_data['status'].isin([1, 2]))
        ]
    temp = pick_data[['order_accept_date', 'total_amount']].copy()
    temp.loc[:, 'order_accept_date'] = pd.to_datetime(temp['order_accept_date'])
    temp.set_index('order_accept_date', inplace=True)
    temp.resample('D').sum().plot()


toggle2 = ToggleButtons(options=area_list)
toggle2.observe(graph_by_area, names='value')
display(toggle2)

ToggleButtons(options=('東京', '神奈川', '千葉', '埼玉', '北関東'), value='東京')